In [18]:
from dataclasses import dataclass
from math import gcd, ceil
import re
import numpy as np
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate, E, W, N, S, winds, TURNLEFT, TURNRIGHT
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst, shoelace
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

inf = float('inf')

In [19]:
def willcoll(x,x2,dx,dx2):
    if x > x2:
        x, x2 = x2, x
        dx, dx2 = dx2, dx
    
    print(x,x2,dx,dx2)
    left = dx - dx2
    right = x2 - x
    if left == 0 or right == 0:
        return False
    t =  right / left
    if t < 0:
        return False
    
    
    print(left, right, t, x + t * dx)
    if minn <= x + t * dx <= maxx:
        return x + t * dx
    else:
        return False
    
willcoll(20,12,-2,-1)

12 20 -1 -2
1 8 8.0 4.0


False

In [20]:
def tonormal(x,dx,y,dy):
    slope = dy / dx
    # Y - y = 2*(X - x)
    X = slope
    c = slope * -x + y
    Y = -1
    return X, Y, c
tonormal(20,-1,25,-2)
    
def poi(a,b,c,a2,b2,c2):
    if any(calc == 0 for calc in [(a*b2 - a2*b), (a*b2 - a2*b)]):
        print([(a*b2 - a2*b), (a*b2 - a2*b)])
        return False
    
    return (b*c2 - b2*c) / (a*b2 - a2*b), (c*a2 - c2*a) / (a*b2 - a2*b)

a,b,c = tonormal(20,-2, 25, -2)
a2, b2, c2 = tonormal(12,-1,31,-2)
poi(a,b,c,a2,b2,c2)


(-2.0, 3.0)

In [21]:
# Hailstone A: 19, 13, 30 @ -2, 1, -2
# Hailstone B: 18, 19, 22 @ -1, -1, -2

a,b,c = tonormal(19,-2, 13, 1)
a2, b2, c2 = tonormal(18,-1,19,-1)
poi(a,b,c,a2,b2,c2)

(14.333333333333334, 15.333333333333334)

In [34]:
f = open("in.txt").read().split("\n")
lines = [line for line in f]

def point_vector_to_general_form_2d(x, y, dx, dy):
    # Coefficients for the general form ax + by + c = 0
    a = dy
    b = -dx
    c = dx*y - dy*x

    return a, b, c

    
def point_vector_to_general_form3d(x, y, z, dx, dy, dz):
    # Coefficients for the general form ax + by + cz + d = 0
    a = dy - dz
    b = dz - dx
    c = dx - dy
    d = dx*y - dy*x + dz*x - dx*z

    return a, b, c, d


minn = 200000000000000
maxx = 400000000000000
# minn = 7
# maxx = 27
ans = 0
for idx, line in enumerate(lines):
    # print(line)
    x,y,z,dx,dy,dz = ints(line)
    a,b,c = tonormal(x,dx,y,dy)
    for line2 in lines[idx+1:]:
        x2,y2,z2,dx2,dy2,dz2 = ints(line2)
        # print(line, line2)
        a2, b2, c2 = tonormal(x2,dx2,y2,dy2)
        a2, b2, c2 = point_vector_to_general_form_2d(x2,y2, dx2, dy2)
        a2, b2, c2 = tonormal(x2,dx2,z2,dz2)
        a2, b2, c2 = point_vector_to_general_form_2d(x2,z2, dx2, dz2)
        
        res = poi(a,b,c,a2,b2,c2)
        if not res:
            print(line)
            print(line2)
        # within the plane
        if res and all(minn < x < maxx for x in res):
            
            # within time
            if (x < res[0] and dx > 0) or (x > res[0] and dx < 0):
                # also within time for the other vector
                if (x2 < res[0] and dx2 > 0) or (x2 > res[0] and dx2 < 0):
            
                    # print(res)
                    ans += 1
                


        # x2, y2, z2, dx2, dy2, dz2 = ints(line2)
        # tx =  willcoll(x,x2,dx,dx2)
        # ty = willcoll(y,y2,dy,dy2)
        # print('points', tx, ty)
        # if tx and ty and tx == ty:
        #     ans += 1
ans

[0.0, 0.0]
290870407080268, 212152928569033, 190675851373188 @ 112, -345, 355
252420367525238, 228569272027247, 375147379981780 @ 112, -33, -345
[0.0, 0.0]
103429181089434, 257736733508054, 87163120363876 @ 184, 106, 247
346785682749368, 262788638240913, 279278190844758 @ -184, -243, -106
[0.0, 0.0]
304705408852603, 348172200054254, 326846851856545 @ -63, 18, -36
367608961555108, 422327369845133, 267470276532008 @ -126, -20, 36
[0.0, 0.0]
274314688466758, 190840498464265, 406468398277548 @ 77, 33, -549
308689864136095, 166080142587319, 244557881109828 @ -21, 15, -9


21159

In [26]:
line1 = (18, 19, 22, -1, -1, -2)
line2 = (20, 25, 34,  -2, -2, -4)
line3 = (18,19,22, 2,6,12)

line1 = ints('301460992753401, 347607367158274, 241485964190314 @ -55, -40, 57')
line2 = ints('376761503791780, 418312416475585, 320913192640500 @ -154, -112, -45')

line1 = ints('290870407080268, 212152928569033, 190675851373188 @ 112, -345, 355')
line2 = ints('252420367525238, 228569272027247, 375147379981780 @ 112, -33, -345')




# line1 = ints('18, 19, 22 @ -1, -1, -2')
# line2 = ints('20, 25, 34 @ -2, -2, -4')

line3 = (line1[0], line1[1], line1[2], *[b-a for a,b in zip(line1[:3], line2[:3])])
line3

(290870407080268,
 212152928569033,
 190675851373188,
 -38450039555030,
 16416343458214,
 184471528608592)

In [27]:
def plane_from_two_lines(line1, line2):
    # Extracting parameters for the first line
    x1, y1, z1, dx1, dy1, dz1 = line1
    # Extracting parameters for the second line
    x2, y2, z2, dx2, dy2, dz2 = line2

    # Calculate the direction vectors of the lines
    v1 = (dx1, dy1, dz1)
    v2 = (dx2, dy2, dz2)

    # Calculate the normal vector of the plane using the cross product
    normal_vector = (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0]
    )

    # Coefficients of the plane equation
    a, b, c = normal_vector
    d = -(a * x1 + b * y1 + c * z1)  # Use any point on the plane (e.g., the first point of the first line)

    return a, b, c, d

# Example usage:

plane_coefficients = plane_from_two_lines(line1, line3)
print("Plane equation coefficients:", plane_coefficients)

Plane equation coefficients: (-69470479297630210, -34310575246197954, -11426633179165382, 29664778622498573188693176632578)


In [ ]:
a = (-7207276949094167, 8660626693947833, -876757270916945, -626060203972995537368184337545)
b = (-69470479297630210, -34310575246197954, -11426633179165382, 29664778622498573188693176632578)

In [265]:
for idx, line in enumerate(lines):
    # print(line)
    line1 = ints(line)
    for line2 in lines[idx+1:]:
        line2 = ints(line2)
        res = plane_from_two_lines(line1, line2)
        if 0 in res:
            print(res)

(0, 30, -105, 20505244349870265)
(0, 11376, 3792, -2852121279313738704)
(0, -7686, 6405, -304567635412205739)
(-54896, 0, 31536, 9264762523312059152)
(930, 0, 1240, -595664019434438660)
(-3036, 0, -1672, 1326428540510628544)
(0, -2355, 8949, -1778939808582779337)
(-1410, -3995, 0, 1438210992661042990)
(8184, -11253, 0, 1444468937938223538)


In [267]:
r = list(range(-200,200))
planes = []
for idx, line in enumerate(lines):
    # print(line)
    line1 = ints(line)
    for x in r:
        for y in r:
            for z in r:
        
            # line3 = (line1[0], line1[1], line1[2], *[b-a for a,b in zip(line1[:3], line2[:3])])
        
                res = plane_from_two_lines(line1, (0,0,0,x,y,z))
                planes.append((res, line1))


KeyboardInterrupt: 

In [268]:
400**3

64000000

In [17]:
len(planes)

NameError: name 'planes' is not defined

In [269]:
# import math

# def normalize_vector(v):
#     magnitude = math.sqrt(sum(component**2 for component in v))
#     return tuple(component / magnitude for component in v)

# def plane_from_two_lines(line1, line2):
#     # Extracting parameters for the first line
#     x1, y1, z1, dx1, dy1, dz1 = line1
#     # Extracting parameters for the second line
#     x2, y2, z2, dx2, dy2, dz2 = line2

#     # Calculate the direction vectors of the lines
#     v1 = (dx1, dy1, dz1)
#     v2 = (dx2, dy2, dz2)

#     # Calculate the normalized normal vector of the plane using the cross product
#     normal_vector = (
#         v1[1] * v2[2] - v1[2] * v2[1],
#         v1[2] * v2[0] - v1[0] * v2[2],
#         v1[0] * v2[1] - v1[1] * v2[0]
#     )
    
#     normal_vector = normalize_vector(normal_vector)

#     # Coefficients of the plane equation
#     a, b, c = normal_vector
#     d = -(a * x1 + b * y1 + c * z1)  # Use any point on the plane (e.g., the first point of the first line)

#     return a, b, c, d

# # Example usage:


# plane_coefficients = plane_from_two_lines(line1, line3)
# print("Plane equation coefficients:", plane_coefficients)


In [270]:
def checkans(a,b):
    t1, loc1 = res[0]
    t2, loc2 = res[1]
    dt = t2 - t1
    dx,dy,dz = [b - a for a,b in zip(loc1, loc2)]
    t1, t2, dt
    if dt == 0:
        return 0
    deltas = t1 / dt
    return sum(a - b * deltas for a,b in zip(loc1, (dx,dy,dz)))



def timeloc(plane, line):
    x,y,z,dx,dy,dz = line
    p1,p2,p3,p4 = plane
    nums = p1 * x + p2 * y + p3 * z + p4
    par = p1 * dx + p2 * dy + p3 * dz
    if not par or not nums:
        # print('parallel')
        t = 0
    else:
        # print(-nums, par)
        t = -nums / par
    loc = x + dx * t, y + dy * t, z + dz * t
    
    return t, loc
# p = plane_coefficients
for p, l1 in planes:
    res = []
    for line in lines:
        if ints(line) != l1:
            res.append((timeloc(p, ints(line))))
        
        # else:
        #     timeloc(p, ints(line))
    if all(t >= 0 and l[0].is_integer() for t, l in res):

        

        ans1 = checkans(a,b)
        
        if ans1.is_integer() and all(checkans(a,b)== ans1 for a,b in zip(res[1:], res[2:])):
            print('OFUND', ans1, res)


KeyboardInterrupt: 

In [14]:
import z3
from z3 import *

X = Real('x')
Y = Real('y')
Z = Real('z')
DX = Real('dx')
DY = Real('dy')
DZ = Real('dz')

s = Tactic('qfnra-nlsat').solver()
for idx, line in enumerate(lines):
    NX, NY, NZ, NDX, NDY, NDZ = ints(line)
    
    exec(f't{str(idx)} = Int("t{str(idx)}")')
    
    t = eval(f't{str(idx)}')
    
    s.add(t >= 0)
    s.add(X + t * DX == NX + t * NDX)
    s.add(Y + t * DY == NY + t * NDY)
    s.add(Z + t * DZ == NZ + t * NDZ)

    
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(318090941338468, 124187623124113, 231363386790708, r, t)

In [16]:
sum([318090941338468, 124187623124113, 231363386790708])

673641951253289

In [15]:
m.eval(X) + 

318090941338468

In [105]:
plane_coefficients

(0, 8, -4, -64)

In [106]:
192308807793543 / 247

778578169204.6276

In [87]:
import numpy as np

def construct_plane(line1_point, line1_direction, line2_point, line2_direction):
    # Find the normal vector of the plane by taking the cross product of direction vectors
    normal_vector = np.cross(line1_direction, line2_direction)

    # Choose a point on one of the lines as a reference point for the plane
    reference_point = line1_point

    return reference_point, normal_vector

def intersection_with_plane(third_line_point, third_line_direction, plane_reference_point, plane_normal_vector):
    # Write the equation of the plane using the normal vector and the reference point
    # Ax + By + Cz + D = 0
    A, B, C = plane_normal_vector
    D = -np.dot(plane_normal_vector, plane_reference_point)

    # Find the intersection point of the third line with the plane
    t = -(np.dot(plane_normal_vector, third_line_point) + D) / np.dot(plane_normal_vector, third_line_direction)
    intersection_point = third_line_point + t * third_line_direction

    return intersection_point

# Example usage:
line1_point = np.array([18, 19, 22])
line1_direction = np.array([-1, -1, -2])

line2_point = np.array([20, 25, 34])
line2_direction = np.array([-2, -2, -4])

third_line_point = np.array([0, 0, 0])
third_line_direction = np.array([1, 1, 1])

# Construct the plane using the two parallel lines
plane_reference_point, plane_normal_vector = construct_plane(line1_point, line1_direction, line2_point, line2_direction)

# Find the intersection point of the third line with the plane
intersection_point = intersection_with_plane(third_line_point, third_line_direction, plane_reference_point, plane_normal_vector)

print("Intersection Point:", intersection_point)


Intersection Point: [nan nan nan]


C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_20668\3265084700.py:19: RuntimeWarning: invalid value encountered in scalar divide
  t = -(np.dot(plane_normal_vector, third_line_point) + D) / np.dot(plane_normal_vector, third_line_direction)


In [89]:
def plane_from_two_lines(line1, line2):
    # Extracting parameters for the first line
    x1, y1, z1, dx1, dy1, dz1 = line1
    # Extracting parameters for the second line
    x2, y2, z2, dx2, dy2, dz2 = line2

    # Calculate the direction vectors of the lines
    v1 = (dx1, dy1, dz1)
    v2 = (dx2, dy2, dz2)

    # Calculate the normal vector of the plane using the cross product
    normal_vector = (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0]
    )

    # Coefficients of the plane equation
    a, b, c = normal_vector
    d = -(a * x1 + b * y1 + c * z1)  # Use any point on the plane (e.g., the first point of the first line)

    return a, b, c, d

# Example usage:
line1 = (1, 2, 3, 4, 5, 6)
line2 = (7, 8, 9, 10, 11, 12)

plane_coefficients = plane_from_two_lines(line1, line2)
print("Plane equation coefficients:", plane_coefficients)


Plane equation coefficients: (-6, 12, -6, 0)


In [115]:
ans = 0
for idx, line in enumerate(lines):
    # print(line)
    x,y,z,dx,dy,dz = ints(line)
    first = point_vector_to_general_form3d(x,y,z,dx,dy,dz)
    for line2 in lines[idx+1:]:
        x2,y2,z2,dx2,dy2,dz2 = ints(line2)
        # print(line, line2)
        second = point_vector_to_general_form3d(x2,y2,z2,dx2,dy2,dz2)
        print(intersecting_point(first, second))
    

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [59]:
def line_of_intersection(plane1, plane2):
    # Extract coefficients for the first plane
    a1, b1, c1, d1 = plane1
    # Extract coefficients for the second plane
    a2, b2, c2, d2 = plane2

    # Calculate the direction vector of the line of intersection
    direction_vector = (
        b1 * c2 - c1 * b2,
        c1 * a2 - a1 * c2,
        a1 * b2 - b1 * a2
    )

    # Choose a point on the line (you can adjust this as needed)
    intersection_point = (303867887720968, 410071000835863, 211451111726208)

    # Return the line passing through the intersection point with the calculated direction vector
    return (*intersection_point, *direction_vector)

# Example usage:
plane1 = (1, 2, 3, 0)
plane2 = (4, 5, 6, 0)

intersection_line = line_of_intersection(plane1, plane2)
print("Line of intersection between planes:", intersection_line)


Line of intersection between planes: (303867887720968, 410071000835863, 211451111726208, -3, 6, -3)


In [60]:
a = (-7207276949094167, 8660626693947833, -876757270916945, -626060203972995537368184337545)
b = (-69470479297630210, -34310575246197954, -11426633179165382, 29664778622498573188693176632578)
arrow = line_of_intersection(a,b)

In [61]:
arrow

(303867887720968,
 410071000835863,
 211451111726208,
 -129043850649876705264600150447736,
 -21446162079670867485269751618344,
 848943705528489938867677333569248)

In [65]:
def intersection_point_of_lines(line1, line2):
    # Extract parameters for the first line
    x1, y1, z1, dx1, dy1, dz1 = line1
    # Extract parameters for the second line
    x2, y2, z2, dx2, dy2, dz2 = line2

    # Solve for parameters t and s in the parametric equations
    # x1 + t * dx1 = x2 + s * dx2
    # y1 + t * dy1 = y2 + s * dy2
    # z1 + t * dz1 = z2 + s * dz2

    # Check if the lines are parallel
    cross_product = (dy1 * dz2 - dz1 * dy2, dz1 * dx2 - dx1 * dz2, dx1 * dy2 - dy1 * dx2)
    if all(component == 0 for component in cross_product):
        raise ValueError("The lines are parallel and do not intersect.")

    # Calculate parameters t and s
    t = ((y2 - y1) * dz2 - (z2 - z1) * dy2) / cross_product[0]
    s = ((y2 - y1) * dz1 - (z2 - z1) * dy1) / cross_product[0]

    # Calculate the intersection point
    intersection_point = (x1 + t * dx1, y1 + t * dy1, z1 + t * dz1)

    return intersection_point

# Example usage:
line1 = (1, 2, 3, 4, 5, 6)
line2 = (7, 8, 9, 10, 11, 12)

def timeatloc(line, dest):
    x, y, z, dx, dy, dz = line
    x2, y2, z2 = dest
    tx = abs(x2 - x) / abs(dx)
    ty = abs(y2 - y) / abs(dy)
    tz = abs(z2 - z) / abs(dz)
    # print(tx,ty,tz)
    # assert tx == ty
    return tx

res = []
for line in lines:
    l = ints(line)
    loc = intersection_point_of_lines(l,arrow)
    t = timeatloc(l, loc)
    res.append((t,loc))
    

In [68]:
def checkans(one, two):
    t1, loc1 = one
    t2, loc2 = two
    dt = t2 - t1
    dx,dy,dz = [b - a for a,b in zip(loc1, loc2)]
    t1, t2, dt
    if dt == 0:
        return 0
    deltas = t1 / dt
    return sum(a - b * deltas for a,b in zip(loc1, (dx,dy,dz)))



def timeloc(plane, line):
    x,y,z,dx,dy,dz = line
    p1,p2,p3,p4 = plane
    nums = p1 * x + p2 * y + p3 * z + p4
    par = p1 * dx + p2 * dy + p3 * dz
    if not par or not nums:
        # print('parallel')
        t = 0
    else:
        # print(-nums, par)
        t = -nums / par
    loc = x + dx * t, y + dy * t, z + dz * t
    
    return t, loc


In [70]:
checkans(res[1],res[2])

1230720408673512.5

In [113]:
eq = []
for idx, line in enumerate(lines):
    # print(line)
    x,y,z,dx,dy,dz = ints(line)
    a,b,c,d = point_vector_to_general_form3d(x,y,z,dx,dy,dz)
    # print(a,b,c,d)
    eq.append((a,b,c,d))
find_intersecting_plane(eq)
        

LinAlgError: Last 2 dimensions of the array must be square

In [98]:
import numpy as np

def find_intersection_point(line1_coefficients, line2_coefficients):
    # Coefficients for line1
    a1, b1, c1, d1 = line1_coefficients

    # Coefficients for line2
    a2, b2, c2, d2 = line2_coefficients

    # Coefficients matrix
    coefficients_matrix = np.array([
        [a1, b1, c1],
        [a2, b2, c2]
    ])

    # Constants vector
    constants_vector = np.array([-d1, -d2])

    # Use least squares to find the solution
    intersection_point, _, _, _ = np.linalg.lstsq(coefficients_matrix, constants_vector, rcond=None)

    return tuple(intersection_point)

# Example usage:
line1_coefficients = (2, -1, 3, 5)
line2_coefficients = (3, 4, -2, -12)

intersection_point = find_intersection_point(line1_coefficients, line2_coefficients)
print(f"The intersection point is {intersection_point}")


The intersection point is (0.6410256410256417, 1.7666666666666677, -1.5051282051282058)


In [81]:
for idx, line in enumerate(lines):
    # print(line)
    x,y,z,dx,dy,dz = ints(line)
    a,b,c,d = point_vector_to_general_form3d(x,y,z,dx,dy,dz)
    res = find_intersection_point((a,b,c,d),(193477744970720.88, -94059556050659.94, -99418188920060.86, 1))
    print(res)

(-496759922579.875, 27152254127051.72, -26655494204471.84)
(10693371724944.156, -584485851923841.4, 573792480198896.2)
(-1338584079262.6562, 73165272475700.27, -71826688396437.62)
(68418690873408.06, -3739677049604596.5, 3671258358731149.5)
(-9112634948940.781, 498084826601339.4, -488972191652398.6)
(-1974942292951.9531, 107947788432692.66, -105972846139740.72)
(-26821150674394.28, 1466009365870146.8, -1439188215195759.5)
(125086491563.70508, -6837065657713.003, 6711979166149.297)
(-4994297195663.219, 272981817732817.38, -267987520537154.28)
(5634920563998.8125, -307997461519947.06, 302362540955948.25)
(5710842241444.0, -312147242100152.75, 306436399858708.3)
(-3814097188311.5625, 208473613540465.94, -204659516352154.47)
(-2381835287805.5, 130188032656507.5, -127806197368702.06)
(116494229862.01758, -6367423758989.018, 6250929529127.001)
(53279666189379.69, -2912197563494728.5, 2858917897305302.0)
(201342465875.85156, -11005118471789.191, 10803776005913.338)
(845838066147.3906, -462324

In [88]:
intersection = (1216609450209.625, -66498297193354.33, 65281687743144.69)
(intersection[0] - x) / dx, (intersection[1] - y) / dy, (intersection[2] - z) / dz

(-773652624871.795, -311543985933.67377, 41994752483.505684)

In [92]:
ints(line)

(192308807793543, 64973264870656, 41722631599898, 247, 422, 561)

In [96]:
intersection = (1216609450209.625, -66498297193354.33, 65281687743144.69)
point= (215462196262294, 360053686369355, 415014825348072)
delta = (36, 7, -133)
dest = (x + dx for x, dx in zip(point, delta))
for x,dx,x2 in zip(point, delta, dest):
    print((x2-x)/dx)

1.0
1.0
1.0
